In [1]:
#Import Splinter and BeautifulSoup for web scraping
from splinter import Browser
from bs4 import BeautifulSoup as soup
#Import pandas for dataframe building & requests/citipy for geolocating
import pandas as pd
import requests
from citipy import citipy
#Import RegEx for cleaning
import re

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
#Visit the sentiment index website
url= 'https://www.sentiment-index.com/most-loved'
browser.visit(url)
#Delay for loading
browser.is_element_present_by_css('div.list_text', wait_time=1)

False

In [4]:
#Set up website for scraping
html=browser.html
#make soup!
travel_soup=soup(html, 'html.parser')

In [5]:
#Identify the location elements
location_elems=travel_soup.findAll('div', class_="VM7gjN")
cities=[]
for locale in location_elems:
    name=locale.text
    cities.append(name)
print(cities)

['100LexingtonKentucky, United States23.38Tourism\nSentiment\nScore®Restaurants & DiningTOP ASSET:VISIT\n\n\n\n\n\n97San DiegoCalifornia, United States23.54Tourism\nSentiment\nScore®BeachesTOP ASSET:VISIT\n\n\n\n\n\n94Cook Islands\u200b23.84Tourism\nSentiment\nScore®BeachesTOP ASSET:VISIT\n\n\n\n\n\n91Park CityUtah, United States24.13Tourism\nSentiment\nScore®Skiing & SnowboardingTOP ASSET:VISIT\n\n\n\n\n\n88Newcastle Upon TyneEngland, United Kingdom24.3Tourism\nSentiment\nScore®Restaurants & DiningTOP ASSET:VISIT\n\n\n\n\n\n85BrisbaneQueensland, Australia24.42Tourism\nSentiment\nScore®Restaurants & DiningTOP ASSET:VISIT\n\n\n\n\n\n82Scenic RimQueensland, Australia24.56Tourism\nSentiment\nScore®Nature PhotographyTOP ASSET:VISIT\n\n\n\n\n\n79Big Bear LakeCalifornia, United States24.72Tourism\nSentiment\nScore®Nature PhotographyTOP ASSET:VISIT\n\n\n\n\n\n76GrampiansVictoria, Australia24.95Tourism\nSentiment\nScore®Hiking & Rock ClimbingTOP ASSET:VISIT\n\n\n\n\n\n73FernieBritish Columbia,

In [6]:
#Check how the list(s) populated
len(cities)

3

this indicates that each column on the page is an item in the 'list'

In [7]:
#Make the three location elements that are in 'cities' into actual lists
split_items=[]
list_of_cities=[]
for city in cities:
    #Seperate the elements
    split_items=city.split('\n\n\n\n\n\n')
    for item in split_items:
        #Use regex to crop out just the locale name
        citystate=re.findall("^\D*\d+(?:[.,]\d+)*(\D+)\d",item)
        list_of_cities.append(citystate)
print(list_of_cities)

[['LexingtonKentucky, United States'], ['San DiegoCalifornia, United States'], ['Cook Islands\u200b'], ['Park CityUtah, United States'], ['Newcastle Upon TyneEngland, United Kingdom'], ['BrisbaneQueensland, Australia'], ['Scenic RimQueensland, Australia'], ['Big Bear LakeCalifornia, United States'], ['GrampiansVictoria, Australia'], ['FernieBritish Columbia, Canada'], ['DubrovnikCroatia'], ['DurangoColorado, United States'], ['Port StephensNew South Wales, Australia'], ['Antigua and Barbuda\u200b'], ['DubaiUnited Arab Emirates'], ['Isle of WightEngland, United Kingdom'], ['Turks and Caicos Islands\u200b'], ['MarbellaAndalusia, Spain'], ['SavannahGeorgia, United States'], ['Port Douglas & DaintreeQueensland, Australia'], ['Cape TownWestern Cape, South Africa'], ['Magnetic IslandQueensland, Australia'], ['StellenboschWestern Cape, South Africa'], ['Byron BayNew South Wales, Australia'], ['Great Ocean RoadVictoria, Australia'], ['QueenstownOtago, New Zealand'], ['Palm SpringsCalifornia, U

In [8]:
#We want a list of items, not a list of lists
final_list = []
for each in list_of_cities:
    final_list.extend(each)
print(final_list)

['LexingtonKentucky, United States', 'San DiegoCalifornia, United States', 'Cook Islands\u200b', 'Park CityUtah, United States', 'Newcastle Upon TyneEngland, United Kingdom', 'BrisbaneQueensland, Australia', 'Scenic RimQueensland, Australia', 'Big Bear LakeCalifornia, United States', 'GrampiansVictoria, Australia', 'FernieBritish Columbia, Canada', 'DubrovnikCroatia', 'DurangoColorado, United States', 'Port StephensNew South Wales, Australia', 'Antigua and Barbuda\u200b', 'DubaiUnited Arab Emirates', 'Isle of WightEngland, United Kingdom', 'Turks and Caicos Islands\u200b', 'MarbellaAndalusia, Spain', 'SavannahGeorgia, United States', 'Port Douglas & DaintreeQueensland, Australia', 'Cape TownWestern Cape, South Africa', 'Magnetic IslandQueensland, Australia', 'StellenboschWestern Cape, South Africa', 'Byron BayNew South Wales, Australia', 'Great Ocean RoadVictoria, Australia', 'QueenstownOtago, New Zealand', 'Palm SpringsCalifornia, United States', 'PembrokeshireWales, United Kingdom', 

In [9]:
#This should be 100 items, since it was scraped from a top one hundred list. Let's make sure:
len(final_list)

99

In [10]:
#Number 1 is formatted differently on the webpage, so we'll need to capture that and add to the list
Addlastcity=travel_soup.find('div', id="comp-ldasrvr9")
#Make sure we got it
print(Addlastcity.text)

Maldives


In [11]:
#append Maldives to list
final_list.append(Addlastcity)

In [12]:
#Quit browsing session
browser.quit()

In [13]:
#Make series into a DataFrame
top_100=pd.DataFrame({'City':final_list})
top_100.head(5)

,City
0,"LexingtonKentucky, United States"
1,"San DiegoCalifornia, United States"
2,Cook Islands​
3,"Park CityUtah, United States"
4,"Newcastle Upon TyneEngland, United Kingdom"


In [14]:
#Cleaning data
#strip the \u200b off items 
top_100["City"] = top_100["City"].str.replace('\u200b',"")
#get the country split out into its own column
top_100[['City','Country']]=top_100['City'].str.split(',',expand=True)
top_100.head()

,City,Country
0,LexingtonKentucky,United States
1,San DiegoCalifornia,United States
2,Cook Islands,None
3,Park CityUtah,United States
4,Newcastle Upon TyneEngland,United Kingdom


In [15]:
#Now we'll use regex again to identify lowerUpper instances, add a delimiter and then make state/province column
top_100['City']=top_100['City'].str.replace(r'([a-z])([A-Z])',r'\1,\2')
top_100.head()

,City,Country
0,"Lexington,Kentucky",United States
1,"San Diego,California",United States
2,Cook Islands,None
3,"Park City,Utah",United States
4,"Newcastle Upon Tyne,England",United Kingdom


In [16]:
top_100[['City','State/Province']]=top_100['City'].str.split(',',expand=True)
top_100.head()

,City,Country,State/Province
0,Lexington,United States,Kentucky
1,San Diego,United States,California
2,Cook Islands,None,None
3,Park City,United States,Utah
4,Newcastle Upon Tyne,United Kingdom,England


In [17]:
#Load workcation csv for second list of cities
file_path='../Resources/best_cities_for_a_workation.csv'
work_data=pd.read_csv(file_path)
work_data.head(10)

,Ranking,City,Country,Remote connection: Average WiFi speed (Mbps per second),Co-working spaces: Number of co-working spaces,Caffeine: Average price of buying a coffee,Travel: Average price of taxi (per km),After-work drinks: Average price for 2 beers in a bar,Accommodation: Average price of 1 bedroom apartment per month,"Food: Average cost of a meal at a local, mid-level restaurant",Climate: Average number of sunshine hours,Tourist attractions: Number of ‘Things to do’ on Tripadvisor,Instagramability: Number of photos with #
0,1,Bangkok,Thailand,28,117,1.56,0.82,3.08,415.18,1.54,2624,2262,28386616
1,2,New Delhi,India,12,165,1.42,0.19,2.90,179.25,2.90,2685,2019,28528249
2,3,Lisbon,Portugal,33,95,1.56,0.40,3.42,736.19,7.69,2806,1969,10205538
3,4,Barcelona,Spain,37,136,1.59,1.01,5.12,768.46,10.25,2591,2739,62894055
4,5,Buenos Aires,Argentina,17,67,1.22,0.47,2.16,229.55,5.15,2525,1660,21293975
5,5,Budapest,Hungary,37,40,1.20,0.72,2.40,366.66,4.81,1988,1468,14267880
6,7,Mumbai,India,23,152,1.57,0.22,3.28,419.64,2.90,2584,892,47201552
7,8,Istanbul,Turkey,13,69,1.23,0.29,3.34,230.10,2.92,2218,2088,116213193
8,9,Bucharest,Romania,54,46,1.78,0.35,2.78,352.35,6.07,2115,726,3376251
9,10,Phuket,Thailand,23,11,1.73,0.76,3.92,301.08,2.72,3450,1698,10190220


In [18]:
#Merge DFs into one 
Cities_DF=top_100.merge(work_data, on='City',how='outer')
Cities_DF.head(15)

,City,Country_x,State/Province,Ranking,Country_y,Remote connection: Average WiFi speed (Mbps per second),Co-working spaces: Number of co-working spaces,Caffeine: Average price of buying a coffee,Travel: Average price of taxi (per km),After-work drinks: Average price for 2 beers in a bar,Accommodation: Average price of 1 bedroom apartment per month,"Food: Average cost of a meal at a local, mid-level restaurant",Climate: Average number of sunshine hours,Tourist attractions: Number of ‘Things to do’ on Tripadvisor,Instagramability: Number of photos with #
0,Lexington,United States,Kentucky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,San Diego,United States,California,40.0,United States,74.0,53.0,3.06,1.34,8.60,1553.79,10.75,3055.0,1118.0,29887093.0
2,Cook Islands,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Park City,United States,Utah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Newcastle Upon Tyne,United Kingdom,England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Brisbane,Australia,Queensland,99.0,Australia,22.0,37.0,2.49,1.17,8.62,929.46,10.78,2968.0,668.0,11948455.0
6,Scenic Rim,Australia,Queensland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Big Bear Lake,United States,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Grampians,Australia,Victoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fernie,Canada,British Columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#Make sure the number of rows reflects adding the lists together
Cities_DF.count()

City                                                             236
Country_x                                                         74
State/Province                                                    88
Ranking                                                          147
Country_y                                                        147
Remote connection: Average WiFi speed (Mbps per second)          147
Co-working spaces: Number of co-working spaces                   147
Caffeine: Average price of buying a coffee                       147
Travel: Average price of taxi (per km)                           147
After-work drinks: Average price for 2 beers in a bar            147
Accommodation: Average price of 1 bedroom apartment per month    147
Food: Average cost of a meal at a local, mid-level restaurant    147
Climate: Average number of sunshine hours                        147
Tourist attractions: Number of ‘Things to do’ on Tripadvisor     147
Instagramability: Number of photos

In [20]:
#Cleaning data
#We want to make a dataframe with JUST the locales to feed into our api calls.
Cities_DF=Cities_DF[['City','State/Province','Country_x','Country_y']]
Cities_DF.rename(columns={'Country_y':'Country'}, inplace=True)
Cities_DF.head()

,City,State/Province,Country_x,Country
0,Lexington,Kentucky,United States,NaN
1,San Diego,California,United States,United States
2,Cook Islands,None,None,NaN
3,Park City,Utah,United States,NaN
4,Newcastle Upon Tyne,England,United Kingdom,NaN


In [21]:
#Country_y had less null values based on the df.count above, so we are using that as our final Country column, but we want to
#use Country_x to fill in all the null values we can before dropping that column
Cities_DF['Country'].fillna(Cities_DF['Country_x'],inplace=True)
Cities_DF.drop('Country_x',axis=1,inplace=True)
Cities_DF.head(30)

,City,State/Province,Country
0,Lexington,Kentucky,United States
1,San Diego,California,United States
2,Cook Islands,None,None
3,Park City,Utah,United States
4,Newcastle Upon Tyne,England,United Kingdom
5,Brisbane,Queensland,Australia
6,Scenic Rim,Queensland,Australia
7,Big Bear Lake,California,United States
8,Grampians,Victoria,Australia
9,Fernie,British Columbia,Canada


In [22]:
#Check our values count and dtypes
Cities_DF.count()
#214 is up from 147

City              236
State/Province     88
Country           214
dtype: int64

In [23]:
Cities_DF.dtypes

City              object
State/Province    object
Country           object
dtype: object

In [24]:
#We need strings
Cities_DF.convert_dtypes().dtypes

City              string
State/Province    string
Country           string
dtype: object

In [25]:
#Above we see Croatia and the UAE in the state column, so I'll move those over
Cities_DF['Country'].fillna(Cities_DF['State/Province'], inplace=True)
Cities_DF.head(30)

,City,State/Province,Country
0,Lexington,Kentucky,United States
1,San Diego,California,United States
2,Cook Islands,None,None
3,Park City,Utah,United States
4,Newcastle Upon Tyne,England,United Kingdom
5,Brisbane,Queensland,Australia
6,Scenic Rim,Queensland,Australia
7,Big Bear Lake,California,United States
8,Grampians,Victoria,Australia
9,Fernie,British Columbia,Canada


In [26]:
#Now it is just Islands that don't have a country. Move Island Name to Country Column 
Cities_DF['Country'].fillna(Cities_DF['City'], inplace=True)
Cities_DF.head(30)

,City,State/Province,Country
0,Lexington,Kentucky,United States
1,San Diego,California,United States
2,Cook Islands,None,Cook Islands
3,Park City,Utah,United States
4,Newcastle Upon Tyne,England,United Kingdom
5,Brisbane,Queensland,Australia
6,Scenic Rim,Queensland,Australia
7,Big Bear Lake,California,United States
8,Grampians,Victoria,Australia
9,Fernie,British Columbia,Canada


In [27]:
#Check the Countries column
countries_list = Cities_DF['Country'].unique()
print(list(countries_list))

[' United States', 'United States', 'Cook Islands', ' United Kingdom', 'Australia', ' Australia', ' Canada', 'Croatia', 'Antigua and Barbuda', 'United Arab Emirates', 'Turks and Caicos Islands', ' Spain', 'South Africa', ' South Africa', ' New Zealand', 'Tanzania', 'Mauritius', 'Italy', 'Cayman Islands', 'Seychelles', 'Morocco', 'France', ' India', 'Barbados', ' Mexico', ' Norway', 'Jamaica', 'Kingdom of the Netherlands', 'Switzerland', ' Italy', 'Greece', ' Denmark', 'Fiji', 'Russia', 'Saint Lucia', 'Bahamas', 'Indonesia', nan, 'Thailand', 'India', 'Portugal', 'Spain', 'Argentina', 'Hungary', 'Turkey', 'Romania', 'Brazil', 'Bulgaria', 'Mexico', 'Vietnam', 'Poland', 'Malaysia', 'Singapore', 'Serbia', 'Czech Republic', 'Cambodia', 'Ukraine', 'Hong Kong', 'Sri Lanka', 'South Korea', 'Chile', 'Germany', 'Japan', 'Austria', 'Colombia', 'Nepal', 'Taiwan', 'China', 'Peru', 'Canada', 'Belarus', 'Slovenia', 'Ecuador', 'United Kingdom', 'Estonia', 'Burma/Myanmar', 'Uruguay', 'Belgium', 'Ireland

In [28]:
#strip out any white space in the city and country columns
Cities_DF["Country"] = Cities_DF["Country"].str.strip()
Cities_DF["City"] = Cities_DF["City"].str.strip()
#Drop the null row
Cities_DF=Cities_DF.dropna(subset=['City'])
#Manually fix the Burma/Myanmar row
Cities_DF.loc[Cities_DF['City'] == 'Yangon', 'Country'] = 'Myanmar'
#Fix Netherlands issue
Cities_DF.loc[Cities_DF['Country'] == 'Kingdom of the Netherlands', 'Country'] = 'Netherlands'

In [ ]:
#Define a function to use on the rows of the DataFrame 
def lat_lng(city, state, country):
    if city is None:
        return None, None
    if state is None:
        return None, None
    location=f'{city},{country}'
    response=requests.get(f"https://nominatim.openstreetmap.org/search?city={city}&country={country}&format=json")
    if response.status_code==200:
        Json=response.json()
        if len(Json) > 0:
            # Extract latitude and longitude coordinates
            lat = float(Json[0]['lat'])
            lng = float(Json[0]['lon'])
            return lat, lng
        else:
            return None, None
    else:
        return None, None

# Apply the lat_lng function to Cities_DF DataFrame to get latitude and longitude coordinates
Cities_DF['Latitude'], Cities_DF['Longitude'] = zip(*Cities_DF.apply(lambda row: lat_lng(row['City'], row['State/Province'], row['Country']), axis=1))

In [ ]:
#Did it work?
Cities_DF.head(10)

In [ ]:
#Looking to see how many rows do not have geolocations 
null_count=Cities_DF['Latitude'].isnull().sum()
print(null_count)

In [ ]:
# Get an easy list of what the nulls are 
Nulls=Cities_DF[Cities_DF['Latitude'].isnull()]
Nulls.head(24)

Sort of. It still didn't give us coordinates for islands and places like "Great Ocean Road", which is a feature and not 
a specified locality. It also is not returning several places it seems like it should. It also highlights an issue with Honolulu.

In [ ]:
#Manually fix the Honolulu row
Cities_DF.loc[Cities_DF['City'] == 'Honolulu', 'Country'] = 'United States'
Cities_DF.loc[Cities_DF['City'] == 'Honolulu', 'State/Province'] = 'Hawaii'

In [ ]:
#redo the nulls df so that it has the updates
Nulls=Cities_DF[Cities_DF['Latitude'].isnull()]
Nulls.head(26)

In [ ]:
#run the same lat-lng function from before, but drop the 'city' part of the request 
#running just country should net us the islands we want. We want to run it on just the nulls to not interfere in the larger
#dataset
def lat_lng(city, state, country):
    if city is None:
        return None, None
    if state is None:
        return None, None
    location=f'{city},{country}'
    response=requests.get(f"https://nominatim.openstreetmap.org/search?country={country}&format=json")
    if response.status_code==200:
        Json=response.json()
        if len(Json) > 0:
            # Extract latitude and longitude coordinates
            lat = float(Json[0]['lat'])
            lng = float(Json[0]['lon'])
            return lat, lng
        else:
            return None, None
    else:
        return None, None
Nulls['Latitude'], Nulls['Longitude'] = zip(*Nulls.apply(lambda row: lat_lng(row['City'], row['State/Province'], row['Country']), axis=1))
Nulls.head(26)

we're seeing here that it's showing the same geolocation for all UK and Australia values, which makes sense since we only called country. These locations are not actually cities, so we are going to drop them. 

In [ ]:
#Drop these incorrect values
cities_to_drop= ['Great Ocean Road', 'Pembrokeshire', 'Cumbria', 'Greater Geelong & the Bellarine', 'Lake District', 'Cornwall',
                'Hunter Valley', 'Perth and Kinross']
Cities_DF = Cities_DF[~Cities_DF['City'].isin(cities_to_drop)]

In [ ]:
#Merge the correct values we  captured onto the greater dataframe
cities_to_merge = ['Honolulu', 'Yangon', "Kaua'i", 'Pizol', 'French Polynesia']
rows_to_merge = Nulls[Nulls['City'].isin(cities_to_merge)]
Cities_DF = pd.merge(Cities_DF, rows_to_merge, on='City', how='left')
Cities_DF.head(5)

In [ ]:
#Fix up the columns again, moving over fillna values
Cities_DF['Latitude_x'].fillna(Cities_DF['Latitude_y'], inplace=True)
Cities_DF['Longitude_x'].fillna(Cities_DF['Longitude_y'], inplace=True)
Cities_DF.rename(columns={'Latitude_x': 'Latitude','Longitude_x':'Longitude','State/Province_x':'State/Province',
                         'Country_x':'Country'},inplace=True)
Cities_DF=Cities_DF[['City','State/Province','Country','Latitude','Longitude']]
Cities_DF.head()

In [ ]:
#redo the nulls df so that it has the updates
Nulls=Cities_DF[Cities_DF['Latitude'].isnull()]
Nulls.head(15)

In [ ]:
#Define a new geolocation fetching function, targeted at the islands we have left.
def get_coordinates(island_name):
    url = f'https://nominatim.openstreetmap.org/search?q={island_name}&format=json&limit=1'
    response = requests.get(url)
    data = response.json()
    if data:
        latitude = data[0]['lat']
        longitude = data[0]['lon']
        return latitude, longitude
    else:
        return None, None
Nulls['Latitude'], Nulls['Longitude'] = zip(*Nulls['Country'].apply(get_coordinates))

In [ ]:
#Check results
Nulls.head(15)

In [ ]:
#Merge DFs into one 
Cities_DF=Cities_DF.merge(Nulls, on='City',how='left')
Cities_DF.head(5) 

In [ ]:
#Fix up the columns again, moving over fillna values
Cities_DF['Latitude_x'].fillna(Cities_DF['Latitude_y'], inplace=True)
Cities_DF['Longitude_x'].fillna(Cities_DF['Longitude_y'], inplace=True)
Cities_DF.rename(columns={'Latitude_x': 'Latitude','Longitude_x':'Longitude','State/Province_x':'State/Province',
                         'Country_x':'Country'},inplace=True)
Cities_DF=Cities_DF[['City','State/Province','Country','Latitude','Longitude']]
Cities_DF.head()

In [ ]:
#check out the the number of nulls now - we should have fixed them all
new_null_count=Cities_DF['Latitude'].isnull().sum()
print(new_null_count)

In [ ]:
#We want to use the index of this DF as the locationID and add a countryID as well for use in the final database.
Cities_DF.reset_index(inplace=True)
Cities_DF.rename(columns={'index': 'LocationID'}, inplace=True)
Cities_DF['CountryID'] = pd.Categorical(Cities_DF['Country']).codes
Cities_DF.head(10)

In [ ]:
#Export list of cities with geo-coordinates to csv
Cities_DF.to_csv('../Resources/CitiesWGeolocation.csv',index=True)